In [1]:
import geopandas as gpd
import networkx as nx
from shapely.geometry import LineString
from shapely.ops import snap

gdf = gpd.read_file("../data/UKR_networks.gpkg", layer="UKR_networks")
gdf.head(2)

,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,geometry
0,4319974,5113,primary,Набережно-Хрещатицька вулиця,None,F,50,0,F,F,"LINESTRING (30.52383 50.46943, 30.52389 50.469..."
1,4327781,5114,secondary,вулиця Ревуцького,None,F,0,0,F,F,"LINESTRING (30.64523 50.41266, 30.64531 50.412..."


In [2]:
# from shapely.geometry import MultiLineString

# # Convert the GeoDataFrame to a list of LineStrings
# lines = gdf.geometry.tolist()

# # Define your tolerance
# tolerance = 0.5  # 2 meters

# # Snap all lines to each other within the tolerance
# snapped_lines = []
# for line in lines:
#     other_lines = MultiLineString([l for l in lines if l != line])
#     snapped_line = snap(line, other_lines, tolerance)
#     snapped_lines.append(snapped_line)

# # Convert the list of snapped LineStrings back to a GeoDataFrame
# gdf['geometry'] = snapped_lines

from geopandas.tools import sjoin

# Create a spatial index
sindex = gdf.sindex

# Define your tolerance
tolerance = 0.5  # 2 meters

# Buffer the roads by the tolerance
roads_buffered = gdf.copy()
roads_buffered.geometry = gdf.geometry.buffer(tolerance)

# Find potential pairs of roads that are within the tolerance of each other
potential_matches_index = list(sindex.intersection(roads_buffered.total_bounds))
potential_matches = gdf.iloc[potential_matches_index]

# Snap the potential matches to each other
snapped = potential_matches.geometry.apply(lambda x: snap(x, potential_matches.unary_union, tolerance))

# Replace the geometry of the roads with the snapped geometry
gdf.geometry = snapped

C:\Users\dkerr\AppData\Local\Temp\ipykernel_28916\1523312394.py:29: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  roads_buffered.geometry = gdf.geometry.buffer(tolerance)


KeyboardInterrupt: 